In [1]:
import pandas as pd
import os
df = pd.read_csv('resources.csv')

In [2]:
df

,reasoning,title,author_or_creator,content_type,tech_field,subfield,programming_languages,technical_tools_and_libraries,summary,difficulty_level,publication_date,estimated_time_to_complete,tags,resource_link,notable_insights
0,The resource is a Java and Object-Oriented Pro...,Java + OOP Concept Cheat Sheet,son9912,Other,Java,Object-Oriented Programming (OOP),Java,"java.util.Scanner, java.lang.Math, java.util.C...",This cheat sheet provides a concise reference ...,Intermediate,"25th September, 2017",Not specified (likely 15-30 minutes for review),"Java, OOP, programming, cheat sheet, object-or...",https://cheatography.com/son9912/cheat-sheets/...,- Practical examples for Java syntax and OOP p...
1,The resource is a structured learning roadmap ...,Complete Roadmap To Learn DSA,GeeksforGeeks,Roadmap,Data Structures and Algorithms,"Data Structures, Algorithms","Python, Java, C++, JavaScript","Standard Template Library (STL) for C++, Java'...",This roadmap provides a structured 5-step guid...,Beginner,"27 Jul, 2025",Not explicitly specified (self-paced),"DSA, Data Structures, Algorithms, Learning Roa...",https://www.geeksforgeeks.org/complete-roadmap...,- Structured 5-step approach: language basics ...
2,The resource is a comprehensive JavaScript tut...,The Modern JavaScript Tutorial,Ilya Kantor,Tutorial,Web Development,JavaScript,JavaScript,None explicitly mentioned,A structured tutorial covering JavaScript fund...,Beginner,"August 5, 2025",Not specified,"JavaScript, Web Development, DOM, OOP, Async P...",https://javascript.info (assumed based on cont...,"Structured three-part approach (language, brow..."
3,The resource describes a free online Java prog...,Java Programming,University of Helsinki (Agile Education Resear...,Other,Programming,Object-Oriented Programming (OOP),Java,"Test My Code, Integrated Development Environme...",A free online course teaching Java programming...,Beginner,Legacy (exact date unspecified),Approximately 5-20 hours per part (14 parts to...,"Java, Programming, OOP, Object-Oriented Progra...",https://www.mooc.fi/en/profile/completions,"- Uses industry-grade IDE from day one, avoidi..."
4,"The resource is a blog post titled ""Best Free ...",Best Free Resources to Learn Golang – Go Codin...,Avdhoot Fulsundar,Blog,Programming,Golang (Go),Go (Golang),None explicitly mentioned beyond Go language i...,This blog post highlights five free resources ...,Beginner,"October 11, 2022",Variable (depends on learner's pace),"Golang, Go, freeCodeCamp, StudyTonight, Effect...",https://www.freecodecamp.org/news/best-free-re...,- Project-based learning with 11+ projects in ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,"The resource is a GitHub issue titled ""Model W...",Model Wishlist,LaurentMazare,Other,AI,LLM,Python,"Hugging Face Transformers, PyTorch",A community-maintained wishlist of machine lea...,Intermediate,2023-10-25,NaN,"AI, LLM, Hugging Face, Model Integration, Mach...",https://github.com/huggingface/candle/issues/1177,- JinaBert's Apache 2 license and alibi bias i...
165,The resource is a GitHub issue (#1018) from th...,Hierarchical Imitation Learning #1018,Dhanushvarma,Other,AI/ML,Reinforcement Learning (RL),Python,Tianshou,A GitHub issue requesting information on exist...,Intermediate,"January 3, 2024",Not specified,"reinforcement-learning, imitation-learning, ti...",https://github.com/thu-ml/tianshou/issues/1018,The issue emphasizes the importance of hierarc...
166,The resource is a blog post by Lequn Chen deta...,Harnessing 3200 Gbps Network: A Journey with R...,Lequn Chen,Blog,High-Performance Networking,"RDMA, High-Performance Computing (HPC)",Not explicitly mentioned,"RDMA, EFA (Elastic Fabric Adapter), libfabric,...",This blog series explores optimizing 3200 Gbps...,Advanced,"Dec 25, 2024",Not specified,"RDMA, EFA, libfabric, High-Performance Network...",https://github.com/abcdabcd987/libfabric-efa-demo,- Achieved 97.1% of 3200 Gbps bandwidth using ...
167,"The resource is a res

In [3]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import sys

# --- Data Preparation ---
def parse_list_column(col):
    return col.fillna('').apply(lambda x: [i.strip() for i in x.split(',')] if x else [])

# Make sure your DataFrame 'df' is loaded here

df['tags_list'] = parse_list_column(df['tags'])
df['languages_list'] = parse_list_column(df['programming_languages'])

# --- Build the Graph ---
G = nx.Graph()
for _, row in df.iterrows():
    title = row['title']
    field = row['tech_field'].strip() if pd.notna(row['tech_field']) else None
    subfield = row['subfield'].strip() if pd.notna(row['subfield']) else None
    tags = row['tags_list']
    langs = row['languages_list']

    G.add_node(title, ntype='resource', label=title, field=field, subfield=subfield)
    for lang in langs:
        G.add_node(lang, ntype='language', label=lang)
        G.add_edge(title, lang, relation='uses_language')
    for tag in tags:
        G.add_node(tag, ntype='tag', label=tag)
        G.add_edge(title, tag, relation='has_tag')
    if subfield:
        G.add_node(subfield, ntype='subfield', label=subfield)
        G.add_edge(title, subfield, relation='in_subfield')
    if field:
        G.add_node(field, ntype='field', label=field)
        if subfield:
            G.add_edge(subfield, field, relation='in_field')

# --- PyVis Visualization ---
net = Network(
    height="900px", width="100%",
    bgcolor="#222222", font_color="white",
    select_menu=True, filter_menu=True
)
net.from_nx(G)

# Assign color by type
color_map = {
    'resource': "#8dc3e9",
    'language': "#ffe066",
    'field': "#b9cc8a",
    'subfield': "#f6abb6",
    'tag': "#f7cac9",
}
for node in net.nodes:
    ntype = node.get('ntype')
    node['color'] = color_map.get(ntype, "#cccccc")

# --- Calculate node size based on number of connections (degree) ---
node_degrees = dict(G.degree())
base_size = 15
size_scale = 3
for node in net.nodes:
    degree = node_degrees.get(node['id'], 0)
    node['size'] = base_size + size_scale * degree

# --- Render and Save as UTF-8 HTML ---
net.show("pyviz_knowledge_network.html", notebook=False)


pyviz_knowledge_network.html


visualization tools list
- purely graph focused
    - pyVis
    - cytoscape 
    - holoviews
    - matplotlib
- multifunctional
    - matplotlib
    - vega altair
    - datashader 
    - plotly
    - seaborn 
    - Bokeh